<a href="https://colab.research.google.com/github/omondistep/Awesome_Python_Scripts/blob/main/garch2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Rolling volatility for forecasting
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf
from arch import arch_model

# Read CSV file from a folder
file_path = r"GARCH_Forecasts_Data.csv"
df = pd.read_csv(file_path)

# Compute returns and drop first observation NAN
# returns_data = df['Price'].pct_change()*100
returns_data = np.log(df['Price'] / df['Price'].shift(1)) * 100
returns_data = returns_data.iloc[1:]
print("Returns Data:")
print(returns_data)

# Get dates associated with returns
returns_dates = df['Date'].iloc[1:len(returns_data) + 1]
print("Returns Dates:")
print(returns_dates)

# Format dates and returns and create dataframe
returns_dates = pd.to_datetime(returns_dates)
returns_dates_df = pd.DataFrame({'Dates': returns_dates})
returns_realized = pd.DataFrame({'Returns': returns_data})
returns_data = pd.DataFrame({
    'Dates': returns_dates_df['Dates'],
    'Returns': returns_realized['Returns']
})
print("Formatted Returns Data:")
print(returns_data)

# Calculate the mean and standard deviation
mean_return = np.mean(returns_data['Returns'])
std_return = np.std(returns_data['Returns'])

# Plot returns, mean return, and plus and minus 1 and 2 standard deviations
plt.figure(figsize=(10, 6))  # Adjust figure size as needed
plt.plot(returns_data['Dates'], returns_data['Returns'], color='navy', linestyle='-', label='Daily Returns')

# Title and axis labels
plt.title("SPY Daily Returns +/- 1 and 2 Standard Deviations (1/1/2000 - 12/31/2024)", fontweight="bold")
plt.xlabel("Date", fontweight="bold")
plt.ylabel("Return", fontweight="bold")

# Add horizontal lines at +1 and -1 standard deviations
plt.axhline(mean_return + 2 * std_return, color='darkred', linestyle='dashed', linewidth=2, label='+2 std')
plt.axhline(mean_return + std_return, color='red', linestyle='dotted', linewidth=2, label='+1 std')
plt.axhline(mean_return, color='black', linestyle='solid', linewidth=2, label='Mean')
plt.axhline(mean_return - std_return, color='red', linestyle='dotted', linewidth=2, label='-1 std')
plt.axhline(mean_return - 2 * std_return, color='darkred', linestyle='dashed', linewidth=2, label='-2 std')

# Customize the plot
plt.grid(True)
plt.legend()
plt.gcf().autofmt_xdate()
plt.show()

# ACF plot of returns and squared returns
# Returns
plt.figure(figsize=(10, 6))  # Adjust figure size as needed
plot_acf(returns_data['Returns'], lags=20, alpha=0.05, color='navy')
plt.title("ACF of SPY Daily Returns (1/1/2000 - 12/31/2024)", fontweight="bold")
plt.xlabel("Lag", fontweight="bold")
plt.ylabel("Autocorrelation", fontweight="bold")
plt.show()

# Squared returns
plt.figure(figsize=(10, 6))  # Adjust figure size as needed
plot_acf(np.power(returns_data['Returns'], 2), lags=20, alpha=0.05, color='navy')
plt.title("ACF of SPY Daily Squared Returns (1/1/2000 - 12/31/2024)", fontweight="bold")
plt.xlabel("Lag", fontweight="bold")
plt.ylabel("Autocorrelation", fontweight="bold")
plt.show()

# Create array to store values
rolling_realized_values = []
rolling_forecast_values = []

# For loop to roll over data and calculate realized monthly volatility
for i in range(1255, len(returns_data) - 21):
    window_data = returns_data['Returns'].iloc[i:i + 22].values
    variance = np.var(window_data, ddof=1)
    rolling_realized_values.append(variance)
print("Rolling Realized Values:")
print(rolling_realized_values)

# For loop to roll over data for GARCH model
for i in range(0, len(returns_data) - 1255 - 21):
    # GARCH model specification GARCH(1,1)
    model = arch_model(returns_data['Returns'].iloc[i:i + 1256].values, mean='Zero',
                      vol='Garch', p=1, q=1, dist='normal')
    results = model.fit(disp='off')
    forecast = results.forecast(horizon=22)
    rolling_forecast_values.append(np.mean(forecast.variance.values[-1, :]))
print("Rolling Forecast Values:")
print(rolling_forecast_values)

# Concatenate dates, realized variance, and forecasted variance
results_dates_array = pd.to_datetime(df['Date'].iloc[1256:len(returns_data) - 20])
results_dates = pd.DataFrame({'Dates': results_dates_array})
results_realized = pd.DataFrame({'Realized Variance': rolling_realized_values})
results_forecasted = pd.DataFrame({'Forecasted Variance': rolling_forecast_values})
results_dates.reset_index(drop=True, inplace=True)
results = pd.DataFrame({
    'Dates': results_dates['Dates'],
    'Realized Variance': results_realized['Realized Variance'],
    'Forecasted Variance': results_forecasted['Forecasted Variance']
})

# Add to results dataframe for annualized one month realized and forecasted standard deviations
results['Realized Volatility'] = np.sqrt(results['Realized Variance']) * np.sqrt(252)
results['Forecasted Volatility'] = np.sqrt(results['Forecasted Variance']) * np.sqrt(252)

# Compute error metrics
results['Error'] = results['Realized Volatility'] - results['Forecasted Volatility']  # a
results['Squared Error'] = results['Error'] ** 2  # b
mse = results['Squared Error'].mean()  # c
rmse = np.sqrt(mse)  # d
positive_pct = (results['Error'] > 0).mean() * 100  # e
negative_pct = (results['Error'] < 0).mean() * 100  # e
avg_pos_err = results.loc[results['Error'] > 0, 'Error'].mean()  # f (positive magnitude)
avg_neg_err = results.loc[results['Error'] < 0, 'Error'].abs().mean()  # f (negative magnitude, absolute)

# Print all computed metrics
print("\n===== Error Metrics =====")
print(f"a. Errors: Computed as 'Realized Volatility - Forecasted Volatility'")
print(f"b. Squared Errors: Computed")
print(f"c. MSE: {mse:.6f}")
print(f"d. RMSE: {rmse:.6f}")
print(f"e. Positive Error %: {positive_pct:.2f}%")
print(f"  Negative Error %: {negative_pct:.2f}%")
print(f"f. Average Positive Error Magnitude: {avg_pos_err:.6f}")
print(f"  Average Negative Error Magnitude: {avg_neg_err:.6f}")

# Save results to file in the same directory
output_path = file_path.rsplit('\\', 1)[0] + '\\GARCH_Forecast_Results.csv'
results_df = results.copy()
results_df.loc[len(results_df)] = ['MSE', mse, '', '', '', '', mse, '']
results_df.loc[len(results_df)] = ['RMSE', '', '', '', '', '', rmse, '']
results_df.to_csv(output_path, index=False)
print(f"\nResults saved to {output_path} with MSE and RMSE included.")

# Copy results to clipboard
print("Results DataFrame:")
print(results)
results.to_clipboard(index=False, sep='\t')
print("\nResults copied to clipboard. Paste into Excel with Ctrl+V.")

# Plot realized versus forecasted volatility
plt.figure(figsize=(10, 6))  # Adjust figure size as needed
plt.plot(results['Dates'], results['Realized Volatility'], color='navy', linestyle='-', label='Realized Volatility')
plt.plot(results['Dates'], results['Forecasted Volatility'], color='darkred', linestyle='--', label='Forecasted Volatility')

# Customize the plot
plt.xlabel('Date', fontweight="bold")
plt.ylabel('Annualized One-Month Volatility', fontweight="bold")
plt.title('Realized versus Forecasted Volatility', fontweight="bold")
plt.grid(True)
plt.legend()
plt.gcf().autofmt_xdate()
plt.show()

ModuleNotFoundError: No module named 'arch'

In [2]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving GARCH_Forecasts_Data.csv to GARCH_Forecasts_Data.csv
User uploaded file "GARCH_Forecasts_Data.csv" with length 111601 bytes
